In [7]:
# 導入必要的套件
import warnings
warnings.filterwarnings('ignore')  # 忽略警告訊息

import pandas as pd  # 資料分析套件
import numpy as np
import os
from time import time
import re

import matplotlib.pyplot as plt # 視覺化套件
import matplotlib
%matplotlib inline
import seaborn as sns  # 視覺化套件


from sklearn.preprocessing import scale  # 標準化操作
from sklearn.model_selection import train_test_split  # 將資料集分成測試集和訓練集
from sklearn.metrics import f1_score  # F1得分

import xgboost as xgb  # XGBoost模型
from sklearn.svm import SVC  ## 支援向量機分類模型
from sklearn.linear_model import LogisticRegression  # 邏輯回歸模型

from sklearn.model_selection import GridSearchCV  # 超參數調參模組
from sklearn.metrics import make_scorer  # 模型評估
import joblib  # 模型的儲存與載入模組

設定變數

In [8]:
# csv_file = 'img_feature.csv'
# csv_file = 'img_feature_no_masked.csv'
# csv_file = 'img_conv2.csv'
csv_file = '../resnet_output.csv'
random_seed = 42
groundtruth_file = '../GroundTruth.csv'

# 1. 資料前處裡

分割test跟train dataset

In [9]:
# 读取 CSV 文件
data = pd.read_csv(csv_file)
groundtruth_data = pd.read_csv(groundtruth_file)
# # 分割数据，例如将数据分为 80% 的训练集和 20% 的测试集
train_data = data.sample(frac=0.8)  # 80% 训练集
test_data = data.drop(train_data.index)  # 剩下的为测试集

# # 分割数据，例如将数据分为 95% 的训练集和 5% 的测试集
# train_data = data.sample(frac=0.95, random_state=random_seed)
# test_data = data.drop(train_data.index)  # 剩下的为测试集

# 将数据保存到新的 CSV 文件中
# train_data.to_csv('train_data.csv', index=False)
# test_data.to_csv('test_data.csv', index=False)

## 1.1轉出x_train, y_train, x_test, y_test

In [10]:
#  將train跟test data轉成x_train跟y_train

x_train = train_data.iloc[:, 1:]
x_test = test_data.iloc[:, 1:]

print(type(x_train))

x_train = x_train.values.tolist()
x_test = x_test.values.tolist()


<class 'pandas.core.frame.DataFrame'>


### 1.1.2 處理y set

In [11]:

# [1617: 'SIC_0025923.jpg', 8128: 'ISIC_0032434.jpg'] -> [1617: 'SIC_0025923', 8128: 'ISIC_0032434']

train_file_name_list = train_data.iloc[:, 0]
test_file_name_list = test_data.iloc[:, 0]

train_file_name_dict = train_file_name_list.to_dict()
test_file_name_dict = test_file_name_list.to_dict()

for idx, (key , value) in enumerate(train_file_name_dict.items()):
  train_file_name_dict[key] = re.sub(r'\.[^.]+$', '', value)
for idx, (key , value) in enumerate(test_file_name_dict.items()):
  test_file_name_dict[key] = re.sub(r'\.[^.]+$', '', value)

train_file_name_list = pd.DataFrame(list(train_file_name_dict.items()), columns=['index', 'Filename'])
test_file_name_list = pd.DataFrame(list(test_file_name_dict.items()), columns=['index', 'Filename'])

# [1617: 'SIC_0025923', 8128: 'ISIC_0032434'] -> [ISIC_0032434 , SIC_0025923]

train_file_name_list = train_file_name_list['Filename'].tolist()
test_file_name_list = test_file_name_list['Filename'].tolist()

# [ISIC_0032434 , SIC_0025923] -> [0,0,0,1] , [1,0,0,0]

y_train = []
for idx, filename in enumerate(train_file_name_list):
  # find file name matched row in ground truth table
  result_df = groundtruth_data[groundtruth_data['image'] == filename]
  row_data_list = result_df.iloc[0, 1:].tolist()
  y_train.append(row_data_list)


y_test = []
for idx, filename in enumerate(test_file_name_list):
  # find file name matched row in ground truth table
  result_df = groundtruth_data[groundtruth_data['image'] == filename]
  row_data_list = result_df.iloc[0, 1:].tolist()
  y_test.append(row_data_list)


In [12]:
# [0,0,0,1] , [1,0,0,0] -> [3 , 0]
y_train = [np.argmax(vector) for vector in y_train]
y_train[0:15]
y_test = [np.argmax(vector) for vector in y_test]
y_test[0:15]

[1, 1, 1, 1, 1, 4, 1, 3, 1, 5, 1, 0, 1, 4, 0]

In [13]:
print(x_train[0:15])
print(y_train[0:15])
print(x_test[0:15])
print(y_test[0:15])

print(type(y_test[0]))

[[0.0688807070255279, 1.4044209718704224, 0.3363117575645447, 0.8357599377632141, 0.4741905033588409, 0.0328936204314231, 0.0, 1.1266499757766724, 0.4635994434356689, 1.6861530542373655, 1.5062901973724363, 0.6703493595123291, 1.3845921754837036, 0.0011254232376813, 0.184741660952568, 0.0, 1.3332502841949463, 0.4062175452709198, 0.6175271272659302, 0.0328866988420486, 0.3121293783187866, 0.1672611683607101, 0.4766384959220886, 0.0621039345860481, 0.9447324872016908, 1.1356393098831177, 0.5925431251525879, 0.5454844236373901, 0.5606906414031982, 1.354729175567627, 0.9326995611190796, 0.2227059006690979, 1.550784468650818, 0.08391884714365, 0.3124170303344726, 0.2356337904930114, 0.1137020140886306, 0.1154835596680641, 2.2315640449523926, 0.1610024124383926, 0.3819605112075805, 0.0865684226155281, 0.1428719609975814, 0.057185523211956, 3.086777687072754, 0.148309126496315, 4.284643173217773, 0.1672021299600601, 2.7159698009490967, 0.390219897031784, 0.0157455075532197, 0.1917713284492492

# 2. 模型建立 & 訓練

In [14]:
from time import time
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error


def train_classifier(clf, X_train, y_train):
    ''' 訓練模型 '''
    # 紀錄訓練時間
    # print("訓練資料 : ".format(X_train[0:4]))
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    print("訓練時間 {:.4f} 秒".format(end - start))

def predict_labels(clf, features, target, evalByMMSE):
    ''' 使用模型進行預測 '''
    # 紀錄預測時間
    start = time()
    if evalByMMSE :
      y_pred = clf.predict_proba(features)  # y_pred 會return probability
      # prob_target = []
      # for vec in target:
      #   prob_target += [fromPermutationToProbability(vec)]
      # prob_pred = []
      # for vec in y_pred:
      #   prob_pred += [fromPermutationToProbability(vec)]
      # target = oneHotVecSerial(len(target))
      # return mean_absolute_error(prob_target, prob_pred) , mean_squared_error(prob_target, prob_pred, squared=False)

    y_pred = clf.predict(features)
    end = time()
    print("預測時間 in {:.4f} 秒".format(end - start))

    return f1_score(target, y_pred, pos_label=1, average='weighted'), sum(target == y_pred) / float(len(y_pred))

def train_predict(clf, X_train, y_train, X_test, y_test , evalByMMSE = False):
    ''' 訓練並評估模型 '''
    # Indicate the classifier and the training set size
    print("訓練 {} 模型，樣本數: {}。".format(clf.__class__.__name__, len(X_train)))
    # 訓練模型
    train_classifier(clf, X_train, y_train)
    # 在訓練集上評估模型
    res1 , res2 = predict_labels(clf, X_train, y_train, evalByMMSE)
    res3, res4 = predict_labels(clf, X_test, y_test, evalByMMSE)
    if evalByMMSE :

      print("訓練集上的 MAE,RMSE : {:.6f} , {:.6f}。".format(res1, res2))

      mae, rmse = predict_labels(clf, X_test, y_test)

      print("測試集上的 MAE,RMSE : {:.6f} , {:.6f}。".format(res3, res4))

      print("different between MSE , RMSE : {:.7f} , {:.7f}".format(mae-0.375 , rmse-0.43301))

    print("訓練集的 F1 score和acc分別為: {:.4f} , {:.4f}。".format(res1 , res2))
    print("測試集的 F1 score和acc分別為: {:.4f} , {:.4f}。".format(res3 , res4))

# def oneHotVecSerial(lenth):
#   y_test = []
#   for i in range(lenth):

#     one_hot_vec = [0] * 24
#     one_hot_vec[i%24] = 1
#     y_test +=  [one_hot_vec]
#   return y_test

# def fromPermutationToProbability(PermutationVector):   # for one vector
#   probVec = [0] * 16
#   for outterIdx in range(24):
#     for innerIdx in range(4):
#       probVec[innerIdx*4 + int(iter_list[outterIdx][innerIdx])] += PermutationVector[outterIdx]

#   return probVec

In [15]:
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import sys 

# 分别建立三個模型
clf_A = LogisticRegression()
clf_B = SVC(kernel='rbf',gamma='auto', probability=True)
clf_C = xgb.XGBClassifier()



# train_predict(clf_A, x_train, y_train, x_test, y_test)
# print('')
# train_predict(clf_B, x_train, y_train, x_test, y_test)
# print('')
train_predict(clf_C, x_train, y_train, x_test, y_test)

clf_C.save_model("XGB_after_conv2_training.json")

print('')

訓練 XGBClassifier 模型，樣本數: 2404。
訓練時間 17.8731 秒
預測時間 in 0.0810 秒
預測時間 in 0.0190 秒
訓練集的 F1 score和acc分別為: 1.0000 , 1.0000。
測試集的 F1 score和acc分別為: 0.7466 , 0.7687。



[train time, eval time, F1 score, acc]

conv2 : []

conv3 : [354.9478 秒, 0.5047 秒, 0.6388 , 0.7024]

conv4 : [72.4374 秒, 0.1140 秒, 0.6227 , 0.6955]


In [16]:
from numpy import loadtxt
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_tree
import matplotlib.pyplot as plt


clf_C = xgb.XGBClassifier()
clf_C.load_model("XGB_after_img_feature_training.json")

graph = xgb.to_graphviz(clf_C, num_trees=1)

# plt.gcf().set_size_inches(18.5, 1.5)
plot_tree(clf_C, num_trees=1)
plt.show()

# plt.show()

# plt.savefig("temp.pdf")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa8 in position 200: invalid start byte